In [1]:
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import shutil
import tarfile

import tensorflow as tf

from auxiliary import values as v
from auxiliary.data.dataset_ht import find_group

import napari
from auxiliary.data.dataset_nuclei import NucleiDataset
from auxiliary.data.dataset_ht import HtDataset, find_group
from auxiliary.data import imaging

from nuclei_segmentation.processing.preprocessing import Preprocessing
from filtering import cardiac_region as cr

In [2]:
s = '0806_E4'

ds = HtDataset()

nuclei_raw_path, _ = ds.read_specimen(s, 'Nuclei', 'RawImages', verbose=1)
# nuclei_raw = imaging.read_image(nuclei_raw_path)

mem_raw_path, _ = ds.read_specimen(s, 'Membrane', 'RawImages', verbose=1)
# mem_raw = imaging.read_image(mem_raw_path)

lines, _ = ds.read_line(s, verbose=1)
lines = imaging.read_image(lines)



	Found: 20190806_E4_DAPI_decon_0.5.nii.gz
	Found: 20190806_E4_mGFP_decon_0.5.nii.gz
	Found: line_20190806_E4.nii.gz


In [3]:
from scipy.ndimage import rotate

lines_rot = rotate(lines, 90, axes=(0, 2))

In [4]:
myo_line = np.where(lines_rot == 1, 1, 0)
spl_line = np.where(lines_rot == 5, 1, 0)
some_line = np.where(lines_rot == 4, 1, 0)
prox_line = np.where(lines_rot == 6, 1, 0)

In [5]:
pp = Preprocessing([
    'isotropy',
    'norm_percentile',
    'bilateral',
])

nuclei = pp.run(nuclei_raw_path, verbose=1)
mem = pp.run(mem_raw_path, verbose=1)

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr4/RawImages/Nuclei/20190806_E4_DAPI_decon_0.5.nii.gz
Running step: isotropy
	Image resolution: 
	X: 0.8503159880638123 um/px
	Y: 0.8503159880638123 um/px
	Z: 0.799049973487854 um/px
	Resampling factor: 0.9397094249725342
	Original shape: (538, 1024, 1024)
	Resampled shape: (506, 1024, 1024)
Running step: norm_percentile
Running step: bilateral
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr4/RawImages/Membrane/20190806_E4_mGFP_decon_0.5.nii.gz
Running step: isotropy
	Image resolution: 
	X: 0.8503159880638123 um/px
	Y: 0.8503159880638123 um/px
	Z: 0.799049973487854 um/px
	Resampling factor: 0.9397094249725342
	Original shape: (538, 1024, 1024)
	Resampled shape: (506, 1024, 1024)
Running step: norm_percentile
Running step: bilateral


In [6]:
viewer = napari.Viewer()
viewer.add_image(nuclei, name='Nuclei', colormap='blue')
viewer.add_image(mem, name='Membrane', colormap='green')

<Image layer 'Membrane' at 0x7237243415d0>

In [7]:
viewer.add_image(myo_line, name='Myocardium', colormap='red')

<Image layer 'Myocardium' at 0x72371f5bae30>

In [8]:

viewer.add_image(spl_line, name='Splanchnic', colormap='blue')

<Image layer 'Splanchnic' at 0x723726205a20>

In [9]:

viewer.add_image(some_line, name='Somatic', colormap='green')

<Image layer 'Somatic' at 0x723726063940>

In [7]:

# viewer.add_image(prox_line, name='Proximal', colormap='yellow')

<Image layer 'Proximal' at 0x74ebeeb06590>

In [14]:
# imaging.save_nii(myo_line.astype(np.uint8), 'myo_line.nii')
# imaging.save_nii(spl_line.astype(np.uint8), 'spl_line.nii')
# imaging.save_nii(some_line.astype(np.uint8), 'some_line.nii')
# imaging.save_nii(prox_line.astype(np.uint8), 'prox_line.nii')

# imaging.save_nii(nuclei.astype(np.float32), 'nuclei.nii')
# imaging.save_nii(mem.astype(np.float32), 'membrane.nii')